# Get Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from fastai import *
from fastai.tabular import *

In [0]:
path = Path('drive/My Drive/Research Projects/Sensory_Data_Project/')
path_a = Path(path/'Data_All')

In [0]:
train_df = pd.read_csv(path_a/'Combined_Feature_Engineering_try_train.csv')
test_df = pd.read_csv(path_a/'Combined_Feature_Engineering_try_test.csv')

I will be rerunning the experiments using Label Smoothing and various feature engineerings

# Functions

In [0]:
from fastai.callbacks import *
from fastai.callback import *

In [0]:
def calcError(arr:np.array):
  mean = arr.mean()
  stds = []
  for _, val in enumerate(arr):
    stds.append((mean-val)**2)
  stds = np.array(stds).sum()
  var = math.sqrt(stds/(len(arr)-1))
  return var/math.sqrt(len(arr))

In [0]:
!git clone https://github.com/sdoria/mish

Cloning into 'mish'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 46 (delta 21), reused 19 (delta 6), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [0]:
%cd mish
from rangerlars import *
from mish import *
from mxresnet import *
from ranger import *
%cd ..

/content/mish
Mish activation loaded...
/content


In [0]:
def flattenAnneal(learn:Learner, lr:float, n_epochs:int, start_pct:float):
  n = len(learn.data.train_dl)
  anneal_start = int(n*n_epochs*start_pct)
  anneal_end = int(n*n_epochs) - anneal_start
  phases = [TrainingPhase(anneal_start).schedule_hp('lr', lr),
           TrainingPhase(anneal_end).schedule_hp('lr', lr, anneal=annealing_cos)]
  sched = GeneralScheduler(learn, phases)
  learn.callbacks.append(sched)
  learn.fit(n_epochs)

In [0]:
def calc_mean(l): return reduce(lambda x, y: x + y, l) / len(l)

In [0]:
def train_model(varlist:list, train:pd.DataFrame, test:pd.DataFrame, is_cel:bool=False):
  dep_var = 'Activity'
  mostconf = []
  valid_avg = []
  test_avg = []
  for _, col in enumerate(varlist):
    train[col] = train[col].astype('float64')
    train[col] = train[col].fillna(train[col].mean())
    test[col] = test[col].astype('float64')
    test[col] = test[col].fillna(train[col].mean())

  procs = [Categorify, Normalize]

  for x in range(10):
    data = (TabularList.from_df(train, path = '', cat_names = [], cont_names = varlist, procs = procs)
         .split_by_rand_pct()
         .label_from_df(cols=dep_var)
         .databunch(bs=500))
    learn = tabular_learner(data, layers=[1000,500], metrics=accuracy, loss_func=LabelSmoothingCrossEntropy(), opt_func=partial(Ranger, betas=(0.95,0.99), eps=1e-6))
    flattenAnneal(learn, 4e-3, 5, 0.72)
    valid_avg.append(float(learn.validate()[1]))
    data_test = (TabularList.from_df(test, path = '', cat_names = [], cont_names = varlist, procs = procs, processor=data.processor)
         .split_none()
         .label_from_df(cols=dep_var))
    
    data_test.valid = data_test.train
    data_test = data_test.databunch()
    learn.data.valid_dl = data_test.valid_dl
    test_avg.append(float(learn.validate()[1]))
    
    interp = ClassificationInterpretation.from_learner(learn)
    mostconf.append(interp.confusion_matrix())

    print(f'Validation Accuracy {x+1}: {valid_avg[x]}')
    print(f'Test Accuracy {x+1}: {test_avg[x]}')


  print(f'Average Validation Accuracy: {calc_mean(valid_avg)}')
  print(f'Validation Error: {calcError(np.array(valid_avg))}')
  print(f'Average Test Accuracy: {calc_mean(test_avg)}')
  print(f'Test Error: {calcError(np.array(test_avg))}')
  
  return valid_avg, test_avg, mostconf

# Base

In [0]:
cont_vars = ['altimeterRelativeAltitude(m)', 'accelerometerAccelerationX(G)',
 'accelerometerAccelerationY(G)', 'accelerometerAccelerationZ(G)',
 'motionQuaternionX(R)', 'motionQuaternionY(R)', 'motionQuaternionZ(R)',
 'motionQuaternionW(R)', 'motionGravityX(G)', 'motionGravityY(G)',
 'motionGravityZ(G)', 'motionYaw(rad)', 'motionRoll(rad)', 'motionPitch(rad)',
 'motionUserAccelerationX(G)', 'motionUserAccelerationY(G)', 'motionUserAccelerationZ(G)',
 'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)']

dep_var = 'Activity'

In [0]:
data = (TabularList.from_df(train_df, path = '', cat_names = [], cont_names = cont_vars, procs = [Categorify, FillMissing, Normalize])
         .split_by_rand_pct()
         .label_from_df(cols=dep_var)
         .databunch(bs=500))
data_test = (TabularList.from_df(test_df, path='', cat_names=[], cont_names = cont_vars, procs=[Categorify, FillMissing, Normalize], processor=data.processor)
.split_none()
.label_from_df(cols=dep_var))
data_test.valid = data_test.train
data_test = data_test.databunch()

## CEL

In [0]:
val_avg1, test_avg1, most_conf1 = train_model(cont_vars, train_df, test_df, True)

Validation Accuracy 10: 0.9635579586029053
Test Accuracy 10: 0.8569731116294861
Average Validation Accuracy: 0.960743635892868
Validation Error: 0.0011278836536060093
Average Test Accuracy: 0.8533310055732727
Test Error: 0.0027264493774702223


## LSCE

In [0]:
val_avg, test_avg, most_conf = train_model(data, data_test)

Validation Accuracy 10: 0.8540370464324951
Test Accuracy 10: 0.8540370464324951
Average Validation Accuracy: 0.8616509675979614
Validation Error: 0.011166760050293907
Average Test Accuracy: 0.8519119262695313
Test Error: 0.0031088961592907554


# Base + Time Variation

In [0]:
cont_vars = ['altimeterRelativeAltitude(m)', 'accelerometerAccelerationX(G)',
 'accelerometerAccelerationY(G)', 'accelerometerAccelerationZ(G)',
 'motionQuaternionX(R)', 'motionQuaternionY(R)', 'motionQuaternionZ(R)',
 'motionQuaternionW(R)', 'motionGravityX(G)', 'motionGravityY(G)',
 'motionGravityZ(G)', 'motionYaw(rad)', 'motionRoll(rad)', 'motionPitch(rad)',
 'motionUserAccelerationX(G)', 'motionUserAccelerationY(G)', 'motionUserAccelerationZ(G)',
 'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)',
 'accelerometerAccelerationX(G) prev 1', 'accelerometerAccelerationX(G) prev 2',
 'accelerometerAccelerationX(G) prev 3', 'accelerometerAccelerationX(G) prev 4', 
 'accelerometerAccelerationX(G) prev 5', 'accelerometerAccelerationY(G) prev 1',
 'accelerometerAccelerationY(G) prev 2', 'accelerometerAccelerationY(G) prev 3',
 'accelerometerAccelerationY(G) prev 4', 'accelerometerAccelerationY(G) prev 5',
 'accelerometerAccelerationZ(G) prev 1', 'accelerometerAccelerationZ(G) prev 2',
 'accelerometerAccelerationZ(G) prev 3', 'accelerometerAccelerationZ(G) prev 4',
 'accelerometerAccelerationZ(G) prev 5', 'motionYaw(rad) prev 1',
 'motionYaw(rad) prev 2', 'motionYaw(rad) prev 3', 'motionYaw(rad) prev 4',
 'motionYaw(rad) prev 5', 'motionRoll(rad) prev 1', 'motionRoll(rad) prev 2',
 'motionRoll(rad) prev 3', 'motionRoll(rad) prev 4', 'motionRoll(rad) prev 5',
 'motionPitch(rad) prev 1', 'motionPitch(rad) prev 2', 'motionPitch(rad) prev 3',
 'motionPitch(rad) prev 4', 'motionPitch(rad) prev 5', 'gyroRotationX(rad/s) prev 1',
 'gyroRotationX(rad/s) prev 2', 'gyroRotationX(rad/s) prev 3', 'gyroRotationX(rad/s) prev 4',
 'gyroRotationX(rad/s) prev 5', 'gyroRotationY(rad/s) prev 1', 'gyroRotationY(rad/s) prev 2',
 'gyroRotationY(rad/s) prev 3', 'gyroRotationY(rad/s) prev 4', 'gyroRotationY(rad/s) prev 5',
 'gyroRotationZ(rad/s) prev 1', 'gyroRotationZ(rad/s) prev 2', 'gyroRotationZ(rad/s) prev 3',
 'gyroRotationZ(rad/s) prev 4', 'gyroRotationZ(rad/s) prev 5', 'motionUserAccelerationX(G) prev 1',
 'motionUserAccelerationX(G) prev 2', 'motionUserAccelerationX(G) prev 3',
 'motionUserAccelerationX(G) prev 4', 'motionUserAccelerationX(G) prev 5',
 'motionUserAccelerationY(G) prev 1', 'motionUserAccelerationY(G) prev 2',
 'motionUserAccelerationY(G) prev 3', 'motionUserAccelerationY(G) prev 4',
 'motionUserAccelerationY(G) prev 5', 'motionUserAccelerationZ(G) prev 1',
 'motionUserAccelerationZ(G) prev 2', 'motionUserAccelerationZ(G) prev 3',
 'motionUserAccelerationZ(G) prev 4', 'motionUserAccelerationZ(G) prev 5',
 'motionQuaternionX(R) prev 1', 'motionQuaternionX(R) prev 2', 'motionQuaternionX(R) prev 3',
 'motionQuaternionX(R) prev 4', 'motionQuaternionX(R) prev 5', 'motionQuaternionY(R) prev 1',
 'motionQuaternionY(R) prev 2', 'motionQuaternionY(R) prev 3', 'motionQuaternionY(R) prev 4',
 'motionQuaternionY(R) prev 5', 'motionQuaternionZ(R) prev 1', 'motionQuaternionZ(R) prev 2',
 'motionQuaternionZ(R) prev 3', 'motionQuaternionZ(R) prev 4', 'motionQuaternionZ(R) prev 5',
 'motionQuaternionW(R) prev 1', 'motionQuaternionW(R) prev 2', 'motionQuaternionW(R) prev 3',
 'motionQuaternionW(R) prev 4', 'motionQuaternionW(R) prev 5', 'motionGravityX(G) prev 1',
 'motionGravityX(G) prev 2', 'motionGravityX(G) prev 3', 'motionGravityX(G) prev 4',
 'motionGravityX(G) prev 5', 'motionGravityY(G) prev 1', 'motionGravityY(G) prev 2',
 'motionGravityY(G) prev 3', 'motionGravityY(G) prev 4', 'motionGravityY(G) prev 5',
 'motionGravityZ(G) prev 1', 'motionGravityZ(G) prev 2', 'motionGravityZ(G) prev 3',
 'motionGravityZ(G) prev 4', 'motionGravityZ(G) prev 5']

dep_var = 'Activity'

In [0]:
val_avg1, test_avg1, most_conf1 = train_model(cont_vars, train_df, test_df)

Validation Accuracy 10: 0.9746097922325134
Test Accuracy 10: 0.8677385449409485
Average Validation Accuracy: 0.9733319818973541
Validation Error: 0.00037627472960635467
Average Test Accuracy: 0.865872073173523
Test Error: 0.0017185671259813643


In [0]:
val_avg1, test_avg1, most_conf1 = train_model(cont_vars, train_df, test_df, True)

Validation Accuracy 10: 0.9743220806121826
Test Accuracy 10: 0.8796225190162659
Average Validation Accuracy: 0.9732415735721588
Validation Error: 0.0005042629530421646
Average Test Accuracy: 0.869115698337555
Test Error: 0.002144409863583512


# Base + FE

In [0]:
cont_vars = ['altimeterRelativeAltitude(m)', 'accelerometerAccelerationX(G)',
 'accelerometerAccelerationY(G)', 'accelerometerAccelerationZ(G)',
 'motionQuaternionX(R)', 'motionQuaternionY(R)', 'motionQuaternionZ(R)',
 'motionQuaternionW(R)', 'motionGravityX(G)', 'motionGravityY(G)',
 'motionGravityZ(G)', 'motionYaw(rad)', 'motionRoll(rad)', 'motionPitch(rad)',
 'motionUserAccelerationX(G)', 'motionUserAccelerationY(G)', 'motionUserAccelerationZ(G)',
 'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)','SoSaccelerometerAccelerationX(G)',
 'SoSaccelerometerAccelerationY(G)', 'SoSaccelerometerAccelerationZ(G)',
 'SoSmotionQuaternionX(R)', 'SoSmotionQuaternionY(R)', 'SoSmotionQuaternionZ(R)',
 'SoSmotionQuaternionW(R)', 'SoSmotionGravityX(G)', 'SoSmotionGravityY(G)',
 'SoSmotionGravityZ(G)', 'SoSmotionYaw(rad)', 'SoSmotionRoll(rad)', 'SoSmotionPitch(rad)',
 'SoSmotionUserAccelerationX(G)', 'SoSmotionUserAccelerationY(G)',
 'SoSmotionUserAccelerationZ(G)', 'SoSgyroRotationX(rad/s)',
 'SoSgyroRotationY(rad/s)', 'SoSgyroRotationZ(rad/s)', 'RollingaccelerometerAccelerationX(G)mean',
 'RollingaccelerometerAccelerationY(G)mean', 'RollingaccelerometerAccelerationZ(G)mean',
 'RollingmotionQuaternionX(R)mean', 'RollingmotionQuaternionY(R)mean',
 'RollingmotionQuaternionZ(R)mean', 'RollingmotionQuaternionW(R)mean',
 'RollingmotionGravityX(G)mean', 'RollingmotionGravityY(G)mean',
 'RollingmotionGravityZ(G)mean', 'RollingmotionYaw(rad)mean', 'RollingmotionRoll(rad)mean',
 'RollingmotionPitch(rad)mean', 'RollingmotionUserAccelerationX(G)mean',
 'RollingmotionUserAccelerationY(G)mean', 'RollingmotionUserAccelerationZ(G)mean',
 'RollinggyroRotationX(rad/s)mean', 'RollinggyroRotationY(rad/s)mean',
 'RollinggyroRotationZ(rad/s)mean', 'accelerometerAccelerationX(G)Variance',
 'accelerometerAccelerationY(G)Variance', 'accelerometerAccelerationZ(G)Variance',
 'motionQuaternionX(R)Variance', 'motionQuaternionY(R)Variance', 'motionQuaternionZ(R)Variance',
 'motionQuaternionW(R)Variance', 'motionGravityX(G)Variance', 'motionGravityY(G)Variance',
 'motionGravityZ(G)Variance', 'motionYaw(rad)Variance', 'motionRoll(rad)Variance',
 'motionPitch(rad)Variance', 'motionUserAccelerationX(G)Variance', 'motionUserAccelerationY(G)Variance',
 'motionUserAccelerationZ(G)Variance', 'gyroRotationX(rad/s)Variance', 'gyroRotationY(rad/s)Variance',
 'gyroRotationZ(rad/s)Variance', 'accelerometerAccelerationX(G)RMS', 'accelerometerAccelerationY(G)RMS',
 'accelerometerAccelerationZ(G)RMS', 'motionQuaternionX(R)RMS', 'motionQuaternionY(R)RMS',
 'motionQuaternionZ(R)RMS', 'motionQuaternionW(R)RMS', 'motionGravityX(G)RMS',
 'motionGravityY(G)RMS', 'motionGravityZ(G)RMS', 'motionYaw(rad)RMS',
 'motionRoll(rad)RMS', 'motionPitch(rad)RMS', 'motionUserAccelerationX(G)RMS',
 'motionUserAccelerationY(G)RMS', 'motionUserAccelerationZ(G)RMS', 'gyroRotationX(rad/s)RMS',
 'gyroRotationY(rad/s)RMS', 'gyroRotationZ(rad/s)RMS']

dep_var = 'Activity'

In [0]:
!git clone https://github.com/Qwicen/node.git

fatal: destination path 'node' already exists and is not an empty directory.


In [0]:
%cd node/lib
from nn_utils import *
from utils import *

/content/node/lib


In [0]:
class ODST(ModuleWithInit):
    def __init__(self, in_features, num_trees, depth=6, tree_dim=1, flatten_output=True,
                 choice_function=sparsemax, bin_function=sparsemoid,
                 initialize_response_=nn.init.normal_, initialize_selection_logits_=nn.init.uniform_,
                 threshold_init_beta=1.0, threshold_init_cutoff=1.0,
                 ):
        """
        Oblivious Differentiable Sparsemax Trees. http://tinyurl.com/odst-readmore
        One can drop (sic!) this module anywhere instead of nn.Linear
        :param in_features: number of features in the input tensor
        :param num_trees: number of trees in this layer
        :param tree_dim: number of response channels in the response of individual tree
        :param depth: number of splits in every tree
        :param flatten_output: if False, returns [..., num_trees, tree_dim],
            by default returns [..., num_trees * tree_dim]
        :param choice_function: f(tensor, dim) -> R_simplex computes feature weights s.t. f(tensor, dim).sum(dim) == 1
        :param bin_function: f(tensor) -> R[0, 1], computes tree leaf weights
        :param initialize_response_: in-place initializer for tree output tensor
        :param initialize_selection_logits_: in-place initializer for logits that select features for the tree
        both thresholds and scales are initialized with data-aware init (or .load_state_dict)
        :param threshold_init_beta: initializes threshold to a q-th quantile of data points
            where q ~ Beta(:threshold_init_beta:, :threshold_init_beta:)
            If this param is set to 1, initial thresholds will have the same distribution as data points
            If greater than 1 (e.g. 10), thresholds will be closer to median data value
            If less than 1 (e.g. 0.1), thresholds will approach min/max data values.
        :param threshold_init_cutoff: threshold log-temperatures initializer, \in (0, inf)
            By default(1.0), log-remperatures are initialized in such a way that all bin selectors
            end up in the linear region of sparse-sigmoid. The temperatures are then scaled by this parameter.
            Setting this value > 1.0 will result in some margin between data points and sparse-sigmoid cutoff value
            Setting this value < 1.0 will cause (1 - value) part of data points to end up in flat sparse-sigmoid region
            For instance, threshold_init_cutoff = 0.9 will set 10% points equal to 0.0 or 1.0
            Setting this value > 1.0 will result in a margin between data points and sparse-sigmoid cutoff value
            All points will be between (0.5 - 0.5 / threshold_init_cutoff) and (0.5 + 0.5 / threshold_init_cutoff)
        """
        super().__init__()
        self.depth, self.num_trees, self.tree_dim, self.flatten_output = depth, num_trees, tree_dim, flatten_output
        self.choice_function, self.bin_function = choice_function, bin_function
        self.threshold_init_beta, self.threshold_init_cutoff = threshold_init_beta, threshold_init_cutoff

        self.response = nn.Parameter(torch.zeros([num_trees, tree_dim, 2 ** depth]), requires_grad=True)
        initialize_response_(self.response)

        self.feature_selection_logits = nn.Parameter(
            torch.zeros([in_features, num_trees, depth]), requires_grad=True
        )
        initialize_selection_logits_(self.feature_selection_logits)

        self.feature_thresholds = nn.Parameter(
            torch.full([num_trees, depth], float('nan'), dtype=torch.float32), requires_grad=True
        )  # nan values will be initialized on first batch (data-aware init)

        self.log_temperatures = nn.Parameter(
            torch.full([num_trees, depth], float('nan'), dtype=torch.float32), requires_grad=True
        )

        # binary codes for mapping between 1-hot vectors and bin indices
        with torch.no_grad():
            indices = torch.arange(2 ** self.depth)
            offsets = 2 ** torch.arange(self.depth)
            bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
            bin_codes_1hot = torch.stack([bin_codes, 1.0 - bin_codes], dim=-1)
            self.bin_codes_1hot = nn.Parameter(bin_codes_1hot, requires_grad=False)
            # ^-- [depth, 2 ** depth, 2]

    def forward(self, input):
        assert len(input.shape) >= 2
        if len(input.shape) > 2:
            return self.forward(input.view(-1, input.shape[-1])).view(*input.shape[:-1], -1)
        # new input shape: [batch_size, in_features]

        feature_logits = self.feature_selection_logits
        feature_selectors = self.choice_function(feature_logits, dim=0)
        # ^--[in_features, num_trees, depth]

        feature_values = torch.einsum('bi,ind->bnd', input, feature_selectors)
        # ^--[batch_size, num_trees, depth]

        threshold_logits = (feature_values - self.feature_thresholds) * torch.exp(-self.log_temperatures)

        threshold_logits = torch.stack([-threshold_logits, threshold_logits], dim=-1)
        # ^--[batch_size, num_trees, depth, 2]

        bins = self.bin_function(threshold_logits)
        # ^--[batch_size, num_trees, depth, 2], approximately binary

        bin_matches = torch.einsum('btds,dcs->btdc', bins, self.bin_codes_1hot)
        # ^--[batch_size, num_trees, depth, 2 ** depth]

        response_weights = torch.prod(bin_matches, dim=-2)
        # ^-- [batch_size, num_trees, 2 ** depth]

        response = torch.einsum('bnd,ncd->bnc', response_weights, self.response)
        # ^-- [batch_size, num_trees, tree_dim]

        return response.flatten(1, 2) if self.flatten_output else response

    def initialize(self, input, eps=1e-6):
        # data-aware initializer
        assert len(input.shape) == 2
        if input.shape[0] < 1000:
            warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."
                 "To avoid potential problems, run this model on a data batch with at least 1000 data samples."
                 "You can do so manually before training. Use with torch.no_grad() for memory efficiency.")
        with torch.no_grad():
            feature_selectors = self.choice_function(self.feature_selection_logits, dim=0)
            # ^--[in_features, num_trees, depth]

            feature_values = torch.einsum('bi,ind->bnd', input, feature_selectors)
            # ^--[batch_size, num_trees, depth]

            # initialize thresholds: sample random percentiles of data
            percentiles_q = 100 * np.random.beta(self.threshold_init_beta, self.threshold_init_beta,
                                                 size=[self.num_trees, self.depth])
            self.feature_thresholds.data[...] = torch.as_tensor(
                list(map(np.percentile, check_numpy(feature_values.flatten(1, 2).t()), percentiles_q.flatten())),
                dtype=feature_values.dtype, device=feature_values.device
            ).view(self.num_trees, self.depth)

            # init temperatures: make sure enough data points are in the linear region of sparse-sigmoid
            temperatures = np.percentile(check_numpy(abs(feature_values - self.feature_thresholds)),
                                         q=100 * min(1.0, self.threshold_init_cutoff), axis=0)

            # if threshold_init_cutoff > 1, scale everything down by it
            temperatures /= max(1.0, self.threshold_init_cutoff)
            self.log_temperatures.data[...] = torch.log(torch.as_tensor(temperatures) + eps)

    def __repr__(self):
        return "{}(in_features={}, num_trees={}, depth={}, tree_dim={}, flatten_output={})".format(
            self.__class__.__name__, self.feature_selection_logits.shape[0],
            self.num_trees, self.depth, self.tree_dim, self.flatten_output
        )

In [0]:
class myTabularModel(Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs:ListSizes, n_cont:int, out_sz:int, layers:Collection[int], ps:Collection[float]=None,
                 emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False):
        super().__init__()
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        self.embeds = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        sizes = self.get_sizes(layers, out_sz)
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None]
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += [ODST(n_in, 3)]
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        self.layers = nn.Sequential(*layers)

    def get_sizes(self, layers, out_sz):
        return [self.n_emb + self.n_cont] + layers + [out_sz]

    def forward(self, x_cat:Tensor, x_cont:Tensor) -> Tensor:
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        x = self.layers(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x

In [0]:
def mytabular_learner(data:DataBunch, layers:Collection[int], emb_szs:Dict[str,int]=None, metrics=None,
        ps:Collection[float]=None, emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, **learn_kwargs):
    "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
    emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
    model = myTabularModel(emb_szs, len(data.cont_names), out_sz=data.c, layers=layers, ps=ps, emb_drop=emb_drop,
                         y_range=y_range, use_bn=use_bn)
    return Learner(data, model, metrics=metrics, **learn_kwargs)

In [0]:
dep_var = 'Activity'
mostconf = []
valid_avg = []
test_avg = []
for _, col in enumerate(cont_vars):
  train_df[col] = train_df[col].astype('float64')
  train_df[col] = train_df[col].fillna(train_df[col].mean())
  test_df[col] = test_df[col].astype('float64')
  test_df[col] = test_df[col].fillna(train_df[col].mean())

procs = [Categorify, Normalize]
data = (TabularList.from_df(train_df, path = '', cat_names = [], cont_names = cont_vars, procs = procs)
      .split_by_rand_pct()
      .label_from_df(cols=dep_var)
      .databunch(bs=500))

In [0]:
learn = mytabular_learner(data, layers=[], metrics=accuracy)

In [0]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.


RuntimeError: ignored

In [0]:
val_avg2, test_avg2, most_conf2 = train_m_model(cont_vars, train_df, test_df)

epoch,train_loss,valid_loss,accuracy,time


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.


RuntimeError: ignored

In [0]:
val_avg2, test_avg2, most_conf2 = train_model(cont_vars, train_df, test_df, True)

Validation Accuracy 10: 0.9995891451835632
Test Accuracy 10: 0.9453337788581848
Average Validation Accuracy: 0.9997658133506775
Validation Error: 3.1258453604623174e-05
Average Test Accuracy: 0.9708843052387237
Test Error: 0.005602289020352873


# FE + TS

In [0]:
cont_vars = ['altimeterRelativeAltitude(m)', 'accelerometerAccelerationX(G)',
 'accelerometerAccelerationY(G)', 'accelerometerAccelerationZ(G)',
 'motionQuaternionX(R)', 'motionQuaternionY(R)', 'motionQuaternionZ(R)',
 'motionQuaternionW(R)', 'motionGravityX(G)', 'motionGravityY(G)',
 'motionGravityZ(G)', 'motionYaw(rad)', 'motionRoll(rad)', 'motionPitch(rad)',
 'motionUserAccelerationX(G)', 'motionUserAccelerationY(G)', 'motionUserAccelerationZ(G)',
 'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)',
 'accelerometerAccelerationX(G) prev 1', 'accelerometerAccelerationX(G) prev 2',
 'accelerometerAccelerationX(G) prev 3', 'accelerometerAccelerationX(G) prev 4', 
 'accelerometerAccelerationX(G) prev 5', 'accelerometerAccelerationY(G) prev 1',
 'accelerometerAccelerationY(G) prev 2', 'accelerometerAccelerationY(G) prev 3',
 'accelerometerAccelerationY(G) prev 4', 'accelerometerAccelerationY(G) prev 5',
 'accelerometerAccelerationZ(G) prev 1', 'accelerometerAccelerationZ(G) prev 2',
 'accelerometerAccelerationZ(G) prev 3', 'accelerometerAccelerationZ(G) prev 4',
 'accelerometerAccelerationZ(G) prev 5', 'motionYaw(rad) prev 1',
 'motionYaw(rad) prev 2', 'motionYaw(rad) prev 3', 'motionYaw(rad) prev 4',
 'motionYaw(rad) prev 5', 'motionRoll(rad) prev 1', 'motionRoll(rad) prev 2',
 'motionRoll(rad) prev 3', 'motionRoll(rad) prev 4', 'motionRoll(rad) prev 5',
 'motionPitch(rad) prev 1', 'motionPitch(rad) prev 2', 'motionPitch(rad) prev 3',
 'motionPitch(rad) prev 4', 'motionPitch(rad) prev 5', 'gyroRotationX(rad/s) prev 1',
 'gyroRotationX(rad/s) prev 2', 'gyroRotationX(rad/s) prev 3', 'gyroRotationX(rad/s) prev 4',
 'gyroRotationX(rad/s) prev 5', 'gyroRotationY(rad/s) prev 1', 'gyroRotationY(rad/s) prev 2',
 'gyroRotationY(rad/s) prev 3', 'gyroRotationY(rad/s) prev 4', 'gyroRotationY(rad/s) prev 5',
 'gyroRotationZ(rad/s) prev 1', 'gyroRotationZ(rad/s) prev 2', 'gyroRotationZ(rad/s) prev 3',
 'gyroRotationZ(rad/s) prev 4', 'gyroRotationZ(rad/s) prev 5', 'motionUserAccelerationX(G) prev 1',
 'motionUserAccelerationX(G) prev 2', 'motionUserAccelerationX(G) prev 3',
 'motionUserAccelerationX(G) prev 4', 'motionUserAccelerationX(G) prev 5',
 'motionUserAccelerationY(G) prev 1', 'motionUserAccelerationY(G) prev 2',
 'motionUserAccelerationY(G) prev 3', 'motionUserAccelerationY(G) prev 4',
 'motionUserAccelerationY(G) prev 5', 'motionUserAccelerationZ(G) prev 1',
 'motionUserAccelerationZ(G) prev 2', 'motionUserAccelerationZ(G) prev 3',
 'motionUserAccelerationZ(G) prev 4', 'motionUserAccelerationZ(G) prev 5',
 'motionQuaternionX(R) prev 1', 'motionQuaternionX(R) prev 2', 'motionQuaternionX(R) prev 3',
 'motionQuaternionX(R) prev 4', 'motionQuaternionX(R) prev 5', 'motionQuaternionY(R) prev 1',
 'motionQuaternionY(R) prev 2', 'motionQuaternionY(R) prev 3', 'motionQuaternionY(R) prev 4',
 'motionQuaternionY(R) prev 5', 'motionQuaternionZ(R) prev 1', 'motionQuaternionZ(R) prev 2',
 'motionQuaternionZ(R) prev 3', 'motionQuaternionZ(R) prev 4', 'motionQuaternionZ(R) prev 5',
 'motionQuaternionW(R) prev 1', 'motionQuaternionW(R) prev 2', 'motionQuaternionW(R) prev 3',
 'motionQuaternionW(R) prev 4', 'motionQuaternionW(R) prev 5', 'motionGravityX(G) prev 1',
 'motionGravityX(G) prev 2', 'motionGravityX(G) prev 3', 'motionGravityX(G) prev 4',
 'motionGravityX(G) prev 5', 'motionGravityY(G) prev 1', 'motionGravityY(G) prev 2',
 'motionGravityY(G) prev 3', 'motionGravityY(G) prev 4', 'motionGravityY(G) prev 5',
 'motionGravityZ(G) prev 1', 'motionGravityZ(G) prev 2', 'motionGravityZ(G) prev 3',
 'motionGravityZ(G) prev 4', 'motionGravityZ(G) prev 5', 'SoSaccelerometerAccelerationX(G)',
 'SoSaccelerometerAccelerationY(G)', 'SoSaccelerometerAccelerationZ(G)',
 'SoSmotionQuaternionX(R)', 'SoSmotionQuaternionY(R)', 'SoSmotionQuaternionZ(R)',
 'SoSmotionQuaternionW(R)', 'SoSmotionGravityX(G)', 'SoSmotionGravityY(G)',
 'SoSmotionGravityZ(G)', 'SoSmotionYaw(rad)', 'SoSmotionRoll(rad)', 'SoSmotionPitch(rad)',
 'SoSmotionUserAccelerationX(G)', 'SoSmotionUserAccelerationY(G)',
 'SoSmotionUserAccelerationZ(G)', 'SoSgyroRotationX(rad/s)',
 'SoSgyroRotationY(rad/s)', 'SoSgyroRotationZ(rad/s)', 'RollingaccelerometerAccelerationX(G)mean',
 'RollingaccelerometerAccelerationY(G)mean', 'RollingaccelerometerAccelerationZ(G)mean',
 'RollingmotionQuaternionX(R)mean', 'RollingmotionQuaternionY(R)mean',
 'RollingmotionQuaternionZ(R)mean', 'RollingmotionQuaternionW(R)mean',
 'RollingmotionGravityX(G)mean', 'RollingmotionGravityY(G)mean',
 'RollingmotionGravityZ(G)mean', 'RollingmotionYaw(rad)mean', 'RollingmotionRoll(rad)mean',
 'RollingmotionPitch(rad)mean', 'RollingmotionUserAccelerationX(G)mean',
 'RollingmotionUserAccelerationY(G)mean', 'RollingmotionUserAccelerationZ(G)mean',
 'RollinggyroRotationX(rad/s)mean', 'RollinggyroRotationY(rad/s)mean',
 'RollinggyroRotationZ(rad/s)mean', 'accelerometerAccelerationX(G)Variance',
 'accelerometerAccelerationY(G)Variance', 'accelerometerAccelerationZ(G)Variance',
 'motionQuaternionX(R)Variance', 'motionQuaternionY(R)Variance', 'motionQuaternionZ(R)Variance',
 'motionQuaternionW(R)Variance', 'motionGravityX(G)Variance', 'motionGravityY(G)Variance',
 'motionGravityZ(G)Variance', 'motionYaw(rad)Variance', 'motionRoll(rad)Variance',
 'motionPitch(rad)Variance', 'motionUserAccelerationX(G)Variance', 'motionUserAccelerationY(G)Variance',
 'motionUserAccelerationZ(G)Variance', 'gyroRotationX(rad/s)Variance', 'gyroRotationY(rad/s)Variance',
 'gyroRotationZ(rad/s)Variance', 'accelerometerAccelerationX(G)RMS', 'accelerometerAccelerationY(G)RMS',
 'accelerometerAccelerationZ(G)RMS', 'motionQuaternionX(R)RMS', 'motionQuaternionY(R)RMS',
 'motionQuaternionZ(R)RMS', 'motionQuaternionW(R)RMS', 'motionGravityX(G)RMS',
 'motionGravityY(G)RMS', 'motionGravityZ(G)RMS', 'motionYaw(rad)RMS',
 'motionRoll(rad)RMS', 'motionPitch(rad)RMS', 'motionUserAccelerationX(G)RMS',
 'motionUserAccelerationY(G)RMS', 'motionUserAccelerationZ(G)RMS', 'gyroRotationX(rad/s)RMS',
 'gyroRotationY(rad/s)RMS', 'gyroRotationZ(rad/s)RMS']

dep_var = 'Activity'

In [0]:
val_avg3, test_avg3, most_conf3 = train_model(cont_vars, train_df, test_df)

Validation Accuracy 10: 0.9997124075889587
Test Accuracy 10: 0.984271228313446
Average Validation Accuracy: 0.9996754229068756
Validation Error: 3.089692458032145e-05
Average Test Accuracy: 0.9788535535335541
Test Error: 0.003653229449298378


In [0]:
val_avg3, test_avg3, most_conf3 = train_model(cont_vars, train_df, test_df, True)

Validation Accuracy 10: 0.9997124075889587
Test Accuracy 10: 0.9733659625053406
Average Validation Accuracy: 0.9997041881084442
Validation Error: 3.959409977644437e-05
Average Test Accuracy: 0.9816847264766693
Test Error: 0.001611506805748855


# FE + Mish

## Mish

In [0]:
class MishTabularModel(Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs:ListSizes, n_cont:int, out_sz:int, layers:Collection[int], ps:Collection[float]=None,
                 emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False):
        super().__init__()
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        self.embeds = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        sizes = self.get_sizes(layers, out_sz)
        actns = [mish() for _ in range(len(sizes)-2)] + [None]
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        self.layers = nn.Sequential(*layers)

    def get_sizes(self, layers, out_sz):
        return [self.n_emb + self.n_cont] + layers + [out_sz]

    def forward(self, x_cat:Tensor, x_cont:Tensor) -> Tensor:
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        x = self.layers(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x

In [0]:
def mtabular_learner(data:DataBunch, layers:Collection[int], emb_szs:Dict[str,int]=None, metrics=None,
        ps:Collection[float]=None, emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, **learn_kwargs):
    "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
    emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
    model = MishTabularModel(emb_szs, len(data.cont_names), out_sz=data.c, layers=layers, ps=ps, emb_drop=emb_drop,
                         y_range=y_range, use_bn=use_bn)
    return Learner(data, model, metrics=metrics, **learn_kwargs)

In [0]:
class mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x *( torch.tanh(F.softplus(x)))
        return x

## Training

In [0]:
def train_m_model(varlist:list, train:pd.DataFrame, test:pd.DataFrame, is_cel:bool=False):
  dep_var = 'Activity'
  mostconf = []
  valid_avg = []
  test_avg = []
  for _, col in enumerate(varlist):
    train[col] = train[col].astype('float64')
    train[col] = train[col].fillna(train[col].mean())
    test[col] = test[col].astype('float64')
    test[col] = test[col].fillna(train[col].mean())

  procs = [Categorify, Normalize]

  for x in range(10):
    data = (TabularList.from_df(train, path = '', cat_names = [], cont_names = varlist, procs = procs)
         .split_by_rand_pct()
         .label_from_df(cols=dep_var)
         .databunch(bs=500))
    learn = mtabular_learner(data, layers=[], metrics=accuracy, loss_func=LabelSmoothingCrossEntropy())
    learn.fit(5)
    valid_avg.append(float(learn.validate()[1]))
    data_test = (TabularList.from_df(test, path = '', cat_names = [], cont_names = varlist, procs = procs, processor=data.processor)
         .split_none()
         .label_from_df(cols=dep_var))
    
    data_test.valid = data_test.train
    data_test = data_test.databunch()
    learn.data.valid_dl = data_test.valid_dl
    test_avg.append(float(learn.validate()[1]))
    
    interp = ClassificationInterpretation.from_learner(learn)
    mostconf.append(interp.confusion_matrix())

    print(f'Validation Accuracy {x+1}: {valid_avg[x]}')
    print(f'Test Accuracy {x+1}: {test_avg[x]}')


  print(f'Average Validation Accuracy: {calc_mean(valid_avg)}')
  print(f'Validation Error: {calcError(np.array(valid_avg))}')
  print(f'Average Test Accuracy: {calc_mean(test_avg)}')
  print(f'Test Error: {calcError(np.array(test_avg))}')
  
  return valid_avg, test_avg, mostconf

In [0]:
val_avg4, test_avg4, most_conf4 = train_m_model(cont_vars, train_df, test_df)

Validation Accuracy 10: 0.9997124075889587
Test Accuracy 10: 0.9818944334983826
Average Validation Accuracy: 0.9998027920722962
Validation Error: 3.4540811591539706e-05
Average Test Accuracy: 0.978469067811966
Test Error: 0.004086262370630865


# TS + Mish

In [0]:
cont_vars = ['altimeterRelativeAltitude(m)', 'accelerometerAccelerationX(G)',
 'accelerometerAccelerationY(G)', 'accelerometerAccelerationZ(G)',
 'motionQuaternionX(R)', 'motionQuaternionY(R)', 'motionQuaternionZ(R)',
 'motionQuaternionW(R)', 'motionGravityX(G)', 'motionGravityY(G)',
 'motionGravityZ(G)', 'motionYaw(rad)', 'motionRoll(rad)', 'motionPitch(rad)',
 'motionUserAccelerationX(G)', 'motionUserAccelerationY(G)', 'motionUserAccelerationZ(G)',
 'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)',
 'accelerometerAccelerationX(G) prev 1', 'accelerometerAccelerationX(G) prev 2',
 'accelerometerAccelerationX(G) prev 3', 'accelerometerAccelerationX(G) prev 4', 
 'accelerometerAccelerationX(G) prev 5', 'accelerometerAccelerationY(G) prev 1',
 'accelerometerAccelerationY(G) prev 2', 'accelerometerAccelerationY(G) prev 3',
 'accelerometerAccelerationY(G) prev 4', 'accelerometerAccelerationY(G) prev 5',
 'accelerometerAccelerationZ(G) prev 1', 'accelerometerAccelerationZ(G) prev 2',
 'accelerometerAccelerationZ(G) prev 3', 'accelerometerAccelerationZ(G) prev 4',
 'accelerometerAccelerationZ(G) prev 5', 'motionYaw(rad) prev 1',
 'motionYaw(rad) prev 2', 'motionYaw(rad) prev 3', 'motionYaw(rad) prev 4',
 'motionYaw(rad) prev 5', 'motionRoll(rad) prev 1', 'motionRoll(rad) prev 2',
 'motionRoll(rad) prev 3', 'motionRoll(rad) prev 4', 'motionRoll(rad) prev 5',
 'motionPitch(rad) prev 1', 'motionPitch(rad) prev 2', 'motionPitch(rad) prev 3',
 'motionPitch(rad) prev 4', 'motionPitch(rad) prev 5', 'gyroRotationX(rad/s) prev 1',
 'gyroRotationX(rad/s) prev 2', 'gyroRotationX(rad/s) prev 3', 'gyroRotationX(rad/s) prev 4',
 'gyroRotationX(rad/s) prev 5', 'gyroRotationY(rad/s) prev 1', 'gyroRotationY(rad/s) prev 2',
 'gyroRotationY(rad/s) prev 3', 'gyroRotationY(rad/s) prev 4', 'gyroRotationY(rad/s) prev 5',
 'gyroRotationZ(rad/s) prev 1', 'gyroRotationZ(rad/s) prev 2', 'gyroRotationZ(rad/s) prev 3',
 'gyroRotationZ(rad/s) prev 4', 'gyroRotationZ(rad/s) prev 5', 'motionUserAccelerationX(G) prev 1',
 'motionUserAccelerationX(G) prev 2', 'motionUserAccelerationX(G) prev 3',
 'motionUserAccelerationX(G) prev 4', 'motionUserAccelerationX(G) prev 5',
 'motionUserAccelerationY(G) prev 1', 'motionUserAccelerationY(G) prev 2',
 'motionUserAccelerationY(G) prev 3', 'motionUserAccelerationY(G) prev 4',
 'motionUserAccelerationY(G) prev 5', 'motionUserAccelerationZ(G) prev 1',
 'motionUserAccelerationZ(G) prev 2', 'motionUserAccelerationZ(G) prev 3',
 'motionUserAccelerationZ(G) prev 4', 'motionUserAccelerationZ(G) prev 5',
 'motionQuaternionX(R) prev 1', 'motionQuaternionX(R) prev 2', 'motionQuaternionX(R) prev 3',
 'motionQuaternionX(R) prev 4', 'motionQuaternionX(R) prev 5', 'motionQuaternionY(R) prev 1',
 'motionQuaternionY(R) prev 2', 'motionQuaternionY(R) prev 3', 'motionQuaternionY(R) prev 4',
 'motionQuaternionY(R) prev 5', 'motionQuaternionZ(R) prev 1', 'motionQuaternionZ(R) prev 2',
 'motionQuaternionZ(R) prev 3', 'motionQuaternionZ(R) prev 4', 'motionQuaternionZ(R) prev 5',
 'motionQuaternionW(R) prev 1', 'motionQuaternionW(R) prev 2', 'motionQuaternionW(R) prev 3',
 'motionQuaternionW(R) prev 4', 'motionQuaternionW(R) prev 5', 'motionGravityX(G) prev 1',
 'motionGravityX(G) prev 2', 'motionGravityX(G) prev 3', 'motionGravityX(G) prev 4',
 'motionGravityX(G) prev 5', 'motionGravityY(G) prev 1', 'motionGravityY(G) prev 2',
 'motionGravityY(G) prev 3', 'motionGravityY(G) prev 4', 'motionGravityY(G) prev 5',
 'motionGravityZ(G) prev 1', 'motionGravityZ(G) prev 2', 'motionGravityZ(G) prev 3',
 'motionGravityZ(G) prev 4', 'motionGravityZ(G) prev 5']

dep_var = 'Activity'

In [0]:
val_avg6, test_avg6, most_conf6 = train_m_model(cont_vars, train_df, test_df)

Validation Accuracy 10: 0.9679950475692749
Test Accuracy 10: 0.8585110306739807
Average Validation Accuracy: 0.9678636074066163
Validation Error: 0.00039520025342872805
Average Test Accuracy: 0.865012240409851
Test Error: 0.0014053166455246474


# FE + TS + Mish

In [0]:
cont_vars = ['altimeterRelativeAltitude(m)', 'accelerometerAccelerationX(G)',
 'accelerometerAccelerationY(G)', 'accelerometerAccelerationZ(G)',
 'motionQuaternionX(R)', 'motionQuaternionY(R)', 'motionQuaternionZ(R)',
 'motionQuaternionW(R)', 'motionGravityX(G)', 'motionGravityY(G)',
 'motionGravityZ(G)', 'motionYaw(rad)', 'motionRoll(rad)', 'motionPitch(rad)',
 'motionUserAccelerationX(G)', 'motionUserAccelerationY(G)', 'motionUserAccelerationZ(G)',
 'gyroRotationX(rad/s)', 'gyroRotationY(rad/s)', 'gyroRotationZ(rad/s)',
 'accelerometerAccelerationX(G) prev 1', 'accelerometerAccelerationX(G) prev 2',
 'accelerometerAccelerationX(G) prev 3', 'accelerometerAccelerationX(G) prev 4', 
 'accelerometerAccelerationX(G) prev 5', 'accelerometerAccelerationY(G) prev 1',
 'accelerometerAccelerationY(G) prev 2', 'accelerometerAccelerationY(G) prev 3',
 'accelerometerAccelerationY(G) prev 4', 'accelerometerAccelerationY(G) prev 5',
 'accelerometerAccelerationZ(G) prev 1', 'accelerometerAccelerationZ(G) prev 2',
 'accelerometerAccelerationZ(G) prev 3', 'accelerometerAccelerationZ(G) prev 4',
 'accelerometerAccelerationZ(G) prev 5', 'motionYaw(rad) prev 1',
 'motionYaw(rad) prev 2', 'motionYaw(rad) prev 3', 'motionYaw(rad) prev 4',
 'motionYaw(rad) prev 5', 'motionRoll(rad) prev 1', 'motionRoll(rad) prev 2',
 'motionRoll(rad) prev 3', 'motionRoll(rad) prev 4', 'motionRoll(rad) prev 5',
 'motionPitch(rad) prev 1', 'motionPitch(rad) prev 2', 'motionPitch(rad) prev 3',
 'motionPitch(rad) prev 4', 'motionPitch(rad) prev 5', 'gyroRotationX(rad/s) prev 1',
 'gyroRotationX(rad/s) prev 2', 'gyroRotationX(rad/s) prev 3', 'gyroRotationX(rad/s) prev 4',
 'gyroRotationX(rad/s) prev 5', 'gyroRotationY(rad/s) prev 1', 'gyroRotationY(rad/s) prev 2',
 'gyroRotationY(rad/s) prev 3', 'gyroRotationY(rad/s) prev 4', 'gyroRotationY(rad/s) prev 5',
 'gyroRotationZ(rad/s) prev 1', 'gyroRotationZ(rad/s) prev 2', 'gyroRotationZ(rad/s) prev 3',
 'gyroRotationZ(rad/s) prev 4', 'gyroRotationZ(rad/s) prev 5', 'motionUserAccelerationX(G) prev 1',
 'motionUserAccelerationX(G) prev 2', 'motionUserAccelerationX(G) prev 3',
 'motionUserAccelerationX(G) prev 4', 'motionUserAccelerationX(G) prev 5',
 'motionUserAccelerationY(G) prev 1', 'motionUserAccelerationY(G) prev 2',
 'motionUserAccelerationY(G) prev 3', 'motionUserAccelerationY(G) prev 4',
 'motionUserAccelerationY(G) prev 5', 'motionUserAccelerationZ(G) prev 1',
 'motionUserAccelerationZ(G) prev 2', 'motionUserAccelerationZ(G) prev 3',
 'motionUserAccelerationZ(G) prev 4', 'motionUserAccelerationZ(G) prev 5',
 'motionQuaternionX(R) prev 1', 'motionQuaternionX(R) prev 2', 'motionQuaternionX(R) prev 3',
 'motionQuaternionX(R) prev 4', 'motionQuaternionX(R) prev 5', 'motionQuaternionY(R) prev 1',
 'motionQuaternionY(R) prev 2', 'motionQuaternionY(R) prev 3', 'motionQuaternionY(R) prev 4',
 'motionQuaternionY(R) prev 5', 'motionQuaternionZ(R) prev 1', 'motionQuaternionZ(R) prev 2',
 'motionQuaternionZ(R) prev 3', 'motionQuaternionZ(R) prev 4', 'motionQuaternionZ(R) prev 5',
 'motionQuaternionW(R) prev 1', 'motionQuaternionW(R) prev 2', 'motionQuaternionW(R) prev 3',
 'motionQuaternionW(R) prev 4', 'motionQuaternionW(R) prev 5', 'motionGravityX(G) prev 1',
 'motionGravityX(G) prev 2', 'motionGravityX(G) prev 3', 'motionGravityX(G) prev 4',
 'motionGravityX(G) prev 5', 'motionGravityY(G) prev 1', 'motionGravityY(G) prev 2',
 'motionGravityY(G) prev 3', 'motionGravityY(G) prev 4', 'motionGravityY(G) prev 5',
 'motionGravityZ(G) prev 1', 'motionGravityZ(G) prev 2', 'motionGravityZ(G) prev 3',
 'motionGravityZ(G) prev 4', 'motionGravityZ(G) prev 5', 'SoSaccelerometerAccelerationX(G)',
 'SoSaccelerometerAccelerationY(G)', 'SoSaccelerometerAccelerationZ(G)',
 'SoSmotionQuaternionX(R)', 'SoSmotionQuaternionY(R)', 'SoSmotionQuaternionZ(R)',
 'SoSmotionQuaternionW(R)', 'SoSmotionGravityX(G)', 'SoSmotionGravityY(G)',
 'SoSmotionGravityZ(G)', 'SoSmotionYaw(rad)', 'SoSmotionRoll(rad)', 'SoSmotionPitch(rad)',
 'SoSmotionUserAccelerationX(G)', 'SoSmotionUserAccelerationY(G)',
 'SoSmotionUserAccelerationZ(G)', 'SoSgyroRotationX(rad/s)',
 'SoSgyroRotationY(rad/s)', 'SoSgyroRotationZ(rad/s)', 'RollingaccelerometerAccelerationX(G)mean',
 'RollingaccelerometerAccelerationY(G)mean', 'RollingaccelerometerAccelerationZ(G)mean',
 'RollingmotionQuaternionX(R)mean', 'RollingmotionQuaternionY(R)mean',
 'RollingmotionQuaternionZ(R)mean', 'RollingmotionQuaternionW(R)mean',
 'RollingmotionGravityX(G)mean', 'RollingmotionGravityY(G)mean',
 'RollingmotionGravityZ(G)mean', 'RollingmotionYaw(rad)mean', 'RollingmotionRoll(rad)mean',
 'RollingmotionPitch(rad)mean', 'RollingmotionUserAccelerationX(G)mean',
 'RollingmotionUserAccelerationY(G)mean', 'RollingmotionUserAccelerationZ(G)mean',
 'RollinggyroRotationX(rad/s)mean', 'RollinggyroRotationY(rad/s)mean',
 'RollinggyroRotationZ(rad/s)mean', 'accelerometerAccelerationX(G)Variance',
 'accelerometerAccelerationY(G)Variance', 'accelerometerAccelerationZ(G)Variance',
 'motionQuaternionX(R)Variance', 'motionQuaternionY(R)Variance', 'motionQuaternionZ(R)Variance',
 'motionQuaternionW(R)Variance', 'motionGravityX(G)Variance', 'motionGravityY(G)Variance',
 'motionGravityZ(G)Variance', 'motionYaw(rad)Variance', 'motionRoll(rad)Variance',
 'motionPitch(rad)Variance', 'motionUserAccelerationX(G)Variance', 'motionUserAccelerationY(G)Variance',
 'motionUserAccelerationZ(G)Variance', 'gyroRotationX(rad/s)Variance', 'gyroRotationY(rad/s)Variance',
 'gyroRotationZ(rad/s)Variance', 'accelerometerAccelerationX(G)RMS', 'accelerometerAccelerationY(G)RMS',
 'accelerometerAccelerationZ(G)RMS', 'motionQuaternionX(R)RMS', 'motionQuaternionY(R)RMS',
 'motionQuaternionZ(R)RMS', 'motionQuaternionW(R)RMS', 'motionGravityX(G)RMS',
 'motionGravityY(G)RMS', 'motionGravityZ(G)RMS', 'motionYaw(rad)RMS',
 'motionRoll(rad)RMS', 'motionPitch(rad)RMS', 'motionUserAccelerationX(G)RMS',
 'motionUserAccelerationY(G)RMS', 'motionUserAccelerationZ(G)RMS', 'gyroRotationX(rad/s)RMS',
 'gyroRotationY(rad/s)RMS', 'gyroRotationZ(rad/s)RMS']

dep_var = 'Activity'

In [0]:
val_avg5, test_avg5, most_conf5 = train_m_model(cont_vars, train_df, test_df)

Validation Accuracy 10: 0.999548077583313
Test Accuracy 10: 0.986717939376831
Average Validation Accuracy: 0.9996836483478546
Validation Error: 3.0036892239138067e-05
Average Test Accuracy: 0.9802726328372955
Test Error: 0.0033994817234508823


In [0]:
d

NameError: ignored